In [73]:
import os

from typing import List
from pydantic.dataclasses import dataclass

In [3]:
with open("ALL.GSC", mode="rb") as f:
    data = f.read()
    
len(data)

243698958

In [42]:
def extract_int(data: bytes, offset: int, signed: bool = False):
    return int.from_bytes(data[offset: offset + 4], byteorder="little", signed=signed)

In [60]:
@dataclass
class Entry:
    hash: bytes
    name: str
    offset: int
    size: int
    flags: int
    
    @staticmethod
    def _to_pretty_name(data) -> str:
        b = data
        while b[-1] == 0:
            b = b[:-1]
        return b.decode('ascii')
    
    @staticmethod
    def parse(data: bytes) -> 'Entry':
        return Entry(
            hash = data[0:4],
            name = Entry._to_pretty_name(data[4: 4 + 64]),
            offset = extract_int(data, 68) ^ 0xFF_FF_FF_FF,
            size = extract_int(data, 72),
            flags = data[76]
        )
    
    @staticmethod
    def parse_all(data: bytes, count: int, offset: int) -> List['Entry']:
        sz = Entry.byte_size()
        return [Entry.parse(data[offset + sz * i: offset + sz * (i + 1)]) for i in range(count)]
    
    @staticmethod
    def byte_size() -> int:
        return 4 + 64 + 4 + 4 + 4 + 1

In [64]:
entries_offset = 14
entries_count = extract_int(data, 10)
offset_after_entries = entries_offset + Entry.byte_size() * entries_count

entries_count

5251

In [62]:
entries = Entry.parse_all(data, entries_count, offset = entries_offset) 

entries

[Entry(hash=b'\xac^\xe1\xcb', name='0\\0TO1.XLT', offset=0, size=256, flags=0),
 Entry(hash=b'\xc4\xe1\x03\xa4', name='0\\AGEW_1.PAL', offset=256, size=768, flags=0),
 Entry(hash=b'\xbf\xceI\n', name='0\\AGEW_1BR.GRD', offset=1024, size=8192, flags=0),
 Entry(hash=b'\xca\xe7\xe1\xfd', name='0\\AGEW_1D.GRD', offset=9216, size=8192, flags=0),
 Entry(hash=b'\xbf\xceI\x0c', name='0\\AGEW_1DR.GRD', offset=17408, size=40960, flags=0),
 Entry(hash=b'\xca\xe7\xe1\x0b', name='0\\AGEW_1R.GRD', offset=58368, size=8192, flags=0),
 Entry(hash=b'\xbf\xce>\x1a', name='0\\AGEW_1RG.GRD', offset=66560, size=8192, flags=0),
 Entry(hash=b'\xca\xe7\xe1\x10', name='0\\AGEW_1W.GRD', offset=74752, size=8192, flags=0),
 Entry(hash=b'\xed\xe7\xe1\x0b', name='0\\AGEW_TR.GRD', offset=82944, size=65536, flags=0),
 Entry(hash=b'\xe2\xce+\x1a', name='0\\AGEW_TR4.GRD', offset=148480, size=65536, flags=0),
 Entry(hash=b'\xe7\xd4\xeb\xae', name='0\\ALPHAW.GRD', offset=214016, size=65536, flags=0),
 Entry(hash=b'\xf7\xc

In [72]:
entries[-1].offset + entries[-1].size + offset_after_entries, len(data)

(243698958, 243698958)

In [90]:
def extract_data(e: Entry) -> bytes:
    offset = entry.offset + offset_after_entries
    e_data = bytearray(data[offset: offset + e.size])
    for i in range(len(e_data)):
        e_data[i] = e_data[i] ^ 0x78
    return bytes(e_data)

In [94]:
for entry in entries:
    path_in_name = entry.name.replace('\\', '/')
    path = f"unpacked/{path_in_name}"
    os.makedirs(os.path.dirname(path), exist_ok=True)
    
    with open(path, "wb") as f:
        f.write(extract_data(entry))

In [83]:
# bmp header with xor and without
0x3A ^ 0x42, 0x35 ^ 0x4D

(120, 120)